In [1]:
# 필요한 라이브러리들 (사용 가능한 라이브러리)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [5]:
# 1. LR
# 모델식과 W, b를 직접 정의하고, requires_grad=True로 놓았다.
# ** 기본적인 pytorch의 코드 작성 방법
# 1. x_train, y_train이 있음
# 2. x_train을 선형 조합 OR 기타 등등 식으로 계산하여 prediction을 만드는 식에서,
# ~2. 그 식의 파라미터를 설정(LR의 경우, W와 b를 정의하면 됨)
# 3. optimizer를 설정
# 4. prediction을 이용해 cost function 정의하고,
# ~4. step을 밟아가며, optimizer.zero_grad(), cost.backward(), optimizer.step() 을 수행

In [3]:
# 1. 데이터 준비 : (3, 1). 데이터 하나는 (1,)의 사이즈를 가짐. 
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [4]:
# 2. W, b 설정
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [6]:
# 3. optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

In [15]:
# 4. 반복문에서 prediction 및 cost를 계산하고, optimizer를 업데이트
nb_epochs = 2000
for epoch in range(nb_epochs + 1):
    prediction = x_train * W + b
    cost = torch.mean((prediction-y_train)**2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%200 == 0:
        print(f"Epoch {epoch}, Cost {cost:.4f}, W, b {W.item()}, {b.item()}")

Epoch 0, Cost 2.6664, W, b 0.02675662934780121, 3.9997951984405518
Epoch 200, Cost 0.9156, W, b 0.8913121223449707, 2.520308256149292
Epoch 400, Cost 0.3496, W, b 1.3148983716964722, 1.5573971271514893
Epoch 600, Cost 0.1335, W, b 1.5766483545303345, 0.9623778462409973
Epoch 800, Cost 0.0510, W, b 1.7383946180343628, 0.5946906208992004
Epoch 1000, Cost 0.0195, W, b 1.838343620300293, 0.36748287081718445
Epoch 1200, Cost 0.0074, W, b 1.9001063108444214, 0.2270819991827011
Epoch 1400, Cost 0.0028, W, b 1.9382718801498413, 0.14032267034053802
Epoch 1600, Cost 0.0011, W, b 1.9618557691574097, 0.08671092987060547
Epoch 1800, Cost 0.0004, W, b 1.9764291048049927, 0.05358210206031799
Epoch 2000, Cost 0.0002, W, b 1.9854347705841064, 0.03311028331518173


In [21]:
# 주의 !!!!!!!!!!!! 
# x_train.shape : (3, 1)
# W.shape : (1,)
# x_train.matmul(W) 하면, 왜 (3,) 모양이 되어서 나오는 걸까
x_train @ W

tensor([1.9854, 3.9709, 5.9563], grad_fn=<MvBackward0>)

In [22]:
x_train * W

tensor([[1.9854],
        [3.9709],
        [5.9563]], grad_fn=<MulBackward0>)

In [23]:
# 중간 과정의 shape를 항상 잘 관찰해야 할 듯

In [36]:
# 2. MLR
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 100
for epoch in range(nb_epochs+1):
    prediction = x_train @ W + b
    cost = torch.mean((prediction-y_train)**2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch%20==0:
        print(f"Epoch : {epoch}, Cost : {cost}, W : {W.squeeze().detach()}, b : {b.item():.4f}")

Epoch : 0, Cost : 29661.80078125, W : tensor([0.2940, 0.2936, 0.2902]), b : 0.0034
Epoch : 20, Cost : 5.957088947296143, W : tensor([0.6807, 0.6785, 0.6677]), b : 0.0079
Epoch : 40, Cost : 5.90573787689209, W : tensor([0.6826, 0.6791, 0.6652]), b : 0.0079
Epoch : 60, Cost : 5.854844093322754, W : tensor([0.6844, 0.6797, 0.6627]), b : 0.0080
Epoch : 80, Cost : 5.804478168487549, W : tensor([0.6863, 0.6803, 0.6602]), b : 0.0080
Epoch : 100, Cost : 5.754572868347168, W : tensor([0.6882, 0.6809, 0.6577]), b : 0.0080


In [40]:
# 3. nn.Module을 활용해서.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 데이터 준비
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 작성
class MultilinearRegressionModel(nn.Module):
    def __init__(self):
        super(MultilinearRegressionModel, self).__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

# 학습
model = MultilinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 2000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%200==0:
        print(f"Epoch {epoch} | Cost : {cost:.4f}")

Epoch 0 | Cost : 18750.4551
Epoch 200 | Cost : 6.1172
Epoch 400 | Cost : 5.6906
Epoch 600 | Cost : 5.2945
Epoch 800 | Cost : 4.9267
Epoch 1000 | Cost : 4.5851
Epoch 1200 | Cost : 4.2680
Epoch 1400 | Cost : 3.9734
Epoch 1600 | Cost : 3.6999
Epoch 1800 | Cost : 3.4459
Epoch 2000 | Cost : 3.2099


In [41]:
list(model.parameters())

[Parameter containing:
 tensor([[0.7561, 0.8279, 0.4364]], requires_grad=True),
 Parameter containing:
 tensor([0.3364], requires_grad=True)]

In [43]:
# 6. minibatch & CustomDataset
# Dataset과 DataLoader를 사용해서 학습해 보기.
# TensorDataset을 이용해서 dataset 인스턴스에 바로 담을 수도 있고, 따로 CustomDataset 클래스를 작성해도 된다.
# DataLoader사용법 : Dataset, 그리고 batch size, 그리고 shuffle 인자 (T/F). 

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, TensorDataset, DataLoader

# 데이터 준비
x_train = torch.FloatTensor([[73,  80,  75], 
                             [93,  88,  93], 
                             [89,  91,  80], 
                             [96,  98,  100],   
                             [73,  66,  70]])  
y_train = torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 작성
class MultilinearRegressionModel(nn.Module):
    def __init__(self):
        super(MultilinearRegressionModel, self).__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

# Dataset 작성
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 학습하기
model = MultilinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x, y = samples
        
        prediction = model(x)
        cost = F.mse_loss(prediction, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if epoch%100==0:
            print(f"Epoch:{epoch}, Cost:{cost:.4f}")


Epoch:0, Cost:40952.9766
Epoch:0, Cost:5362.4189
Epoch:0, Cost:4547.7534
Epoch:100, Cost:11.3081
Epoch:100, Cost:31.5804
Epoch:100, Cost:3.9133
Epoch:200, Cost:12.0755
Epoch:200, Cost:25.8850
Epoch:200, Cost:0.4012
Epoch:300, Cost:2.5856
Epoch:300, Cost:2.5749
Epoch:300, Cost:52.3812
Epoch:400, Cost:1.0291
Epoch:400, Cost:18.2857
Epoch:400, Cost:3.9859
Epoch:500, Cost:14.4297
Epoch:500, Cost:12.2114
Epoch:500, Cost:0.0106
Epoch:600, Cost:3.0620
Epoch:600, Cost:0.8864
Epoch:600, Cost:25.5253
Epoch:700, Cost:12.7871
Epoch:700, Cost:2.5543
Epoch:700, Cost:1.4950
Epoch:800, Cost:3.3486
Epoch:800, Cost:1.4299
Epoch:800, Cost:17.1718
Epoch:900, Cost:1.4654
Epoch:900, Cost:9.4852
Epoch:900, Cost:3.8002
Epoch:1000, Cost:1.8445
Epoch:1000, Cost:1.7906
Epoch:1000, Cost:12.3605


In [45]:
list(model.parameters())

[Parameter containing:
 tensor([[0.8215, 0.6957, 0.5244]], requires_grad=True),
 Parameter containing:
 tensor([-0.3066], requires_grad=True)]

In [46]:
# 이번엔 CustomDataset 사용
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

# 모델 작성
class MultilinearRegressionModel(nn.Module):
    def __init__(self):
        super(MultilinearRegressionModel, self).__init__()
        self.linear = nn.Linear(3, 1)
    
    def forward(self, x):
        x = self.linear(x)
        return x

# Dataset 작성
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data  =  torch.FloatTensor([[73,  80,  75], 
                                    [93,  88,  93], 
                                    [89,  91,  80], 
                                    [96,  98,  100],   
                                    [73,  66,  70]])  
        self.y_data  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

# 학습하기
model = MultilinearRegressionModel()
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 2000

for epoch in range(nb_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x, y = samples
        prediction = model(x)
        cost = F.mse_loss(prediction, y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        if epoch%200==0:
            print(f"Epoch : {epoch}, Cost : {cost:.4f}")

Epoch : 0, Cost : 22618.8730
Epoch : 0, Cost : 6950.9160
Epoch : 0, Cost : 2654.0042
Epoch : 200, Cost : 6.3370
Epoch : 200, Cost : 11.5630
Epoch : 200, Cost : 3.8626
Epoch : 400, Cost : 0.3085
Epoch : 400, Cost : 11.0833
Epoch : 400, Cost : 6.4813
Epoch : 600, Cost : 6.2415
Epoch : 600, Cost : 2.8376
Epoch : 600, Cost : 2.9009
Epoch : 800, Cost : 4.2710
Epoch : 800, Cost : 2.0726
Epoch : 800, Cost : 6.8733
Epoch : 1000, Cost : 3.2646
Epoch : 1000, Cost : 1.9684
Epoch : 1000, Cost : 3.1057
Epoch : 1200, Cost : 2.2911
Epoch : 1200, Cost : 3.5613
Epoch : 1200, Cost : 2.5720
Epoch : 1400, Cost : 1.9408
Epoch : 1400, Cost : 1.9342
Epoch : 1400, Cost : 1.7581
Epoch : 1600, Cost : 0.2715
Epoch : 1600, Cost : 1.2632
Epoch : 1600, Cost : 3.3385
Epoch : 1800, Cost : 0.1990
Epoch : 1800, Cost : 0.9911
Epoch : 1800, Cost : 2.6824
Epoch : 2000, Cost : 0.5320
Epoch : 2000, Cost : 1.6380
Epoch : 2000, Cost : 0.6093


In [47]:
list(model.parameters())

[Parameter containing:
 tensor([[1.0187, 0.7518, 0.2609]], requires_grad=True),
 Parameter containing:
 tensor([-0.5519], requires_grad=True)]